In [6]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
import sqlite3  # Using SQLite for immediate start - no setup needed!

# Set random seed for reproducible data
np.random.seed(42)
random.seed(42)

def generate_students(n=1000):
    """Generate synthetic student data"""
    majors = ['Computer Science', 'Business', 'Psychology', 'Biology', 'English', 
              'Mathematics', 'History', 'Chemistry', 'Political Science', 'Art']
    
    students = []
    for i in range(n):
        student_id = f"STU{str(i+1).zfill(6)}"
        first_names = ['Alex', 'Jordan', 'Taylor', 'Morgan', 'Casey', 'Riley', 'Avery', 'Quinn']
        last_names = ['Smith', 'Johnson', 'Williams', 'Brown', 'Jones', 'Garcia', 'Miller', 'Davis']
        
        student = {
            'student_id': student_id,
            'first_name': random.choice(first_names),
            'last_name': random.choice(last_names),
            'email': f"{student_id.lower()}@college.edu",
            'major': random.choice(majors),
            'year_level': random.choice([1, 2, 3, 4]),
            'gpa': round(random.uniform(2.0, 4.0), 2),
            'enrollment_date': datetime(2020, 8, 15) + timedelta(days=random.randint(0, 1460)),
            'is_active': random.choice([True, True, True, False])  # 75% active
        }
        students.append(student)
    
    return pd.DataFrame(students)

def generate_courses(n=50):
    """Generate synthetic course data"""
    departments = ['CS', 'BUS', 'PSY', 'BIO', 'ENG', 'MATH', 'HIST', 'CHEM', 'POLS', 'ART']
    course_types = ['Lecture', 'Lab', 'Seminar', 'Workshop']
    
    courses = []
    for i in range(n):
        dept = random.choice(departments)
        course_num = random.randint(1000, 4999)
        
        course = {
            'course_id': f"{dept}-{course_num}",
            'course_name': f"{dept} Course {course_num}",
            'department': dept,
            'credits': random.choice([1, 2, 3, 4]),
            'course_type': random.choice(course_types),
            'max_enrollment': random.randint(15, 100),
            'semester': random.choice(['Fall', 'Spring', 'Summer']),
            'year': random.choice([2023, 2024, 2025])
        }
        courses.append(course)
    
    return pd.DataFrame(courses)

def generate_enrollments(students_df, courses_df, n=2000):
    """Generate synthetic enrollment data"""
    enrollments = []
    
    for i in range(n):
        student_id = random.choice(students_df['student_id'].tolist())
        course_id = random.choice(courses_df['course_id'].tolist())
        
        # Avoid duplicate enrollments
        if any(e['student_id'] == student_id and e['course_id'] == course_id for e in enrollments):
            continue
            
        enrollment = {
            'enrollment_id': f"ENR{str(i+1).zfill(6)}",
            'student_id': student_id,
            'course_id': course_id,
            'enrollment_date': datetime.now() - timedelta(days=random.randint(1, 365)),
            'grade': random.choice(['A', 'A-', 'B+', 'B', 'B-', 'C+', 'C', 'C-', 'D', 'F', None]),
            'status': random.choice(['Enrolled', 'Completed', 'Dropped', 'Withdrawn'])
        }
        enrollments.append(enrollment)
    
    return pd.DataFrame(enrollments)

def create_database_and_load_data():
    """Create SQLite database and load all data"""
    print("🚀 Generating synthetic campus data...")
    
    # Generate data
    students_df = generate_students(1000)
    courses_df = generate_courses(50)
    enrollments_df = generate_enrollments(students_df, courses_df, 2000)
    
    print(f"✅ Generated:")
    print(f"   - {len(students_df)} students")
    print(f"   - {len(courses_df)} courses") 
    print(f"   - {len(enrollments_df)} enrollments")
    
    # Create database
    conn = sqlite3.connect('campus_data.db')
    
    # Load data into database
    students_df.to_sql('students', conn, if_exists='replace', index=False)
    courses_df.to_sql('courses', conn, if_exists='replace', index=False)
    enrollments_df.to_sql('enrollments', conn, if_exists='replace', index=False)
    
    print("📊 Data loaded into campus_data.db")
    
    # Show some sample queries
    print("\n🔍 Sample Data Analysis:")
    
    # Query 1: Students by major
    query1 = """
    SELECT major, COUNT(*) as student_count, AVG(gpa) as avg_gpa
    FROM students 
    WHERE is_active = 1
    GROUP BY major 
    ORDER BY student_count DESC
    """
    result1 = pd.read_sql_query(query1, conn)
    print("\n📈 Active Students by Major:")
    print(result1.head())
    
    # Query 2: Course enrollment stats
    query2 = """
    SELECT c.department, COUNT(e.enrollment_id) as total_enrollments,
           COUNT(CASE WHEN e.status = 'Completed' THEN 1 END) as completed
    FROM courses c
    LEFT JOIN enrollments e ON c.course_id = e.course_id
    GROUP BY c.department
    ORDER BY total_enrollments DESC
    """
    result2 = pd.read_sql_query(query2, conn)
    print("\n📚 Enrollments by Department:")
    print(result2.head())
    
    # Query 3: Grade distribution
    query3 = """
    SELECT grade, COUNT(*) as count,
           ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM enrollments WHERE grade IS NOT NULL), 1) as percentage
    FROM enrollments 
    WHERE grade IS NOT NULL
    GROUP BY grade 
    ORDER BY count DESC
    """
    result3 = pd.read_sql_query(query3, conn)
    print("\n🎓 Grade Distribution:")
    print(result3)
    
    conn.close()
    
    print("\n🎉 SUCCESS! You now have:")
    print("   - A SQLite database with realistic campus data")
    print("   - Example queries showing data analysis patterns")
    print("   - Foundation for building your portfolio project")
    
    print("\n📝 Next Steps:")
    print("   1. Explore the data with your own SQL queries")
    print("   2. Create visualizations with matplotlib or plotly")
    print("   3. Build a simple API to serve this data")
    print("   4. Add data pipeline automation")

# if __name__ == "__main__":
create_database_and_load_data()

🚀 Generating synthetic campus data...
✅ Generated:
   - 1000 students
   - 50 courses
   - 1953 enrollments
📊 Data loaded into campus_data.db

🔍 Sample Data Analysis:

📈 Active Students by Major:
         major  student_count   avg_gpa
0    Chemistry             79  2.947848
1  Mathematics             78  2.924872
2      English             78  2.917949
3   Psychology             77  2.849870
4     Business             75  2.874933

📚 Enrollments by Department:
  department  total_enrollments  completed
0        BUS                316         69
1         CS                293         83
2       HIST                284         77
3       CHEM                272         68
4       POLS                184         48

🎓 Grade Distribution:
  grade  count  percentage
0     C    205        11.5
1     B    200        11.3
2    C+    182        10.3
3    C-    179        10.1
4    B+    178        10.0
5    A-    174         9.8
6    B-    172         9.7
7     A    170         9.6
8     D   

In [7]:
from sqlalchemy import create_engine, select, insert, update, delete, Text, inspect
from sqlalchemy import  MetaData, Table, Column, Integer, String, Float, DateTime
import pandas as pd
DB_URL = "sqlite:///campus_data.db"
engine = create_engine(DB_URL)
table_names = inspect(engine).get_table_names()
table_names

['courses', 'enrollments', 'students']

In [8]:
metadata = MetaData()
students = Table('students', metadata, autoload_with=engine)

with engine.connect() as connection:
    results = connection.execute(select(students)).mappings()
    # df = results.fetchall()
    student_df = pd.DataFrame(results.fetchall())
student_df

,email,enrollment_date,first_name,gpa,is_active,last_name,major,student_id,year_level
0,stu000001@college.edu,2021-03-12,Jordan,2.45,1,Smith,English,STU000001,2
1,stu000002@college.edu,2023-06-15,Avery,2.44,1,Smith,Computer Science,STU000002,1
2,stu000003@college.edu,2020-08-28,Morgan,3.18,1,Miller,Biology,STU000003,4
3,stu000004@college.edu,2022-07-05,Avery,2.43,1,Garcia,English,STU000004,2
4,stu000005@college.edu,2024-01-03,Jordan,3.69,1,Miller,Business,STU000005,3
...,...,...,...,...,...,...,...,...,...
995,stu000996@college.edu,2022-02-11,Jordan,3.89,1,Brown,Biology,STU000996,2
996,stu000997@college.edu,2023-12-26,Quinn,3.71,1,Miller,Psychology,STU000997,4
997,stu000998@college.edu,2021-06-09,Casey,2.82,1,Brown,History,STU000998,2
998,stu000999@college.edu,2022-09-11,Casey,3.81,0,Jones,Biology,STU000999,4


In [78]:
cols =['first_name', 'last_name','major','gpa','is_active']
filter = 'gpa < 2.5 and `is_active` == 1'
student_df[cols].query(filter).sort_values('gpa')

,first_name,last_name,major,gpa,is_active
386,Jordan,Williams,Chemistry,2.00,1
609,Casey,Miller,English,2.01,1
343,Quinn,Johnson,Psychology,2.01,1
557,Casey,Williams,Psychology,2.01,1
618,Alex,Williams,Chemistry,2.02,1
...,...,...,...,...,...
562,Alex,Johnson,Mathematics,2.48,1
398,Morgan,Davis,History,2.48,1
411,Morgan,Smith,Computer Science,2.49,1
12,Quinn,Williams,English,2.49,1


In [56]:
table_names

['courses', 'enrollments', 'students']

In [61]:
metadata = MetaData()
courses = Table('courses', metadata, autoload_with=engine)

with engine.connect() as connection:
    results = connection.execute(select(courses)).mappings()
    # df = results.fetchall()
    course_df = pd.DataFrame(results.fetchall())
course_df

,course_id,course_name,course_type,credits,department,max_enrollment,semester,year
0,BIO-3921,BIO Course 3921,Workshop,4,BIO,91,Spring,2023
1,BUS-3281,BUS Course 3281,Lecture,1,BUS,57,Fall,2024
2,MATH-3483,MATH Course 3483,Lecture,1,MATH,47,Summer,2025
3,HIST-2239,HIST Course 2239,Seminar,4,HIST,31,Summer,2023
4,ART-1734,ART Course 1734,Lab,3,ART,79,Spring,2024
5,CHEM-1540,CHEM Course 1540,Seminar,1,CHEM,82,Spring,2023
6,ENG-1478,ENG Course 1478,Lecture,3,ENG,31,Spring,2024
7,MATH-4793,MATH Course 4793,Seminar,4,MATH,48,Summer,2024
8,POLS-2282,POLS Course 2282,Lab,4,POLS,55,Summer,2024
9,PSY-3276,PSY Course 3276,Lecture,4,PSY,39,Fall,2023


In [64]:
metadata = MetaData()
enrollment = Table('enrollments', metadata, autoload_with=engine)

with engine.connect() as connection:
    results = connection.execute(select(enrollment)).mappings()
    # df = results.fetchall()
    enrollment_df = pd.DataFrame(results.fetchall())
enrollment_df

,course_id,enrollment_date,enrollment_id,grade,status,student_id
0,BUS-4715,2025-07-09 20:37:54.547022,ENR000001,C-,Dropped,STU000911
1,CS-1477,2025-01-26 20:37:54.547042,ENR000002,A,Completed,STU000460
2,HIST-1208,2025-05-31 20:37:54.547053,ENR000003,C-,Dropped,STU000295
3,POLS-4369,2024-12-21 20:37:54.547063,ENR000004,B,Dropped,STU000449
4,CS-2617,2024-10-05 20:37:54.547072,ENR000005,C-,Completed,STU000944
...,...,...,...,...,...,...
1948,HIST-1208,2024-10-16 20:37:54.645401,ENR001996,B-,Completed,STU000750
1949,CHEM-3102,2025-05-29 20:37:54.645490,ENR001997,A-,Withdrawn,STU000146
1950,PSY-2470,2025-05-17 20:37:54.645578,ENR001998,A,Enrolled,STU000283
1951,BIO-3921,2025-06-30 20:37:54.645666,ENR001999,B,Withdrawn,STU000311


## SQL-Alchemy Session

In [33]:
students.columns.items()

[('student_id', Column('student_id', TEXT(), table=<students>)),
 ('first_name', Column('first_name', TEXT(), table=<students>)),
 ('last_name', Column('last_name', TEXT(), table=<students>)),
 ('email', Column('email', TEXT(), table=<students>)),
 ('major', Column('major', TEXT(), table=<students>)),
 ('year_level', Column('year_level', INTEGER(), table=<students>)),
 ('gpa', Column('gpa', REAL(), table=<students>)),
 ('enrollment_date', Column('enrollment_date', TIMESTAMP(), table=<students>)),
 ('is_active', Column('is_active', INTEGER(), table=<students>))]

In [45]:
from sqlalchemy import create_engine, Boolean
from sqlalchemy.orm import sessionmaker, declarative_base

Base = declarative_base()

class Student(Base):
    __tablename__ = 'students'
    student_id = Column(String, primary_key=True)
    first_name = Column(String)
    last_name = Column(String)
    email = Column(String)
    major = Column(String)
    year_level = Column(Integer)
    gpa = Column(Float)
    enrollment_date = Column(DateTime)
    is_active = Column(Boolean)

    # id =Column(Integer, primary_key=True)
# Replace with your actual database connection string
# engine = create_engine('your_database_connection_string')

# Create all tables in the engine
Base.metadata.create_all(engine)

# Create a session factory
Session = sessionmaker(bind=engine)
# Create a session instance
session = Session()

In [48]:
# Query the Student class
query = session.query(Student).filter(
    Student.gpa < 2.5,
    Student.is_active == True  # OR Student.is_active == 1 depending on your database
)

# Execute the query and get the results
for student in query.all():
    print(f"Name: {student.first_name} {student.last_name}, Major: {student.major}, GPA: {student.gpa}")

Name: Jordan Smith, Major: English, GPA: 2.45
Name: Avery Smith, Major: Computer Science, GPA: 2.44
Name: Avery Garcia, Major: English, GPA: 2.43
Name: Quinn Williams, Major: English, GPA: 2.49
Name: Alex Brown, Major: Art, GPA: 2.17
Name: Quinn Davis, Major: Business, GPA: 2.45
Name: Morgan Brown, Major: Political Science, GPA: 2.28
Name: Quinn Jones, Major: History, GPA: 2.31
Name: Alex Smith, Major: Mathematics, GPA: 2.1
Name: Jordan Brown, Major: Mathematics, GPA: 2.32
Name: Alex Brown, Major: Mathematics, GPA: 2.14
Name: Taylor Johnson, Major: English, GPA: 2.19
Name: Morgan Brown, Major: Psychology, GPA: 2.09
Name: Morgan Williams, Major: Political Science, GPA: 2.1
Name: Quinn Brown, Major: English, GPA: 2.15
Name: Avery Smith, Major: Business, GPA: 2.27
Name: Alex Jones, Major: History, GPA: 2.42
Name: Taylor Johnson, Major: Psychology, GPA: 2.22
Name: Riley Smith, Major: Chemistry, GPA: 2.36
Name: Riley Johnson, Major: Political Science, GPA: 2.38
Name: Casey Johnson, Major: B

In [ ]:
# # df = 
# # You might need to drop the SQLAlchemy internal state column, often '_sa_instance_state'
# if '_sa_instance_state' in df.columns:
#     df = df.drop(columns=['_sa_instance_state'])

# OR index from column 1
pd.DataFrame([s.__dict__ for s in query.all()]).iloc[:, 1:]

,email,student_id,first_name,year_level,gpa,is_active,major,last_name,enrollment_date
0,stu000001@trincoll.edu,STU000001,Jordan,2,2.45,True,English,Smith,2021-03-12
1,stu000002@trincoll.edu,STU000002,Avery,1,2.44,True,Computer Science,Smith,2023-06-15
2,stu000004@trincoll.edu,STU000004,Avery,2,2.43,True,English,Garcia,2022-07-05
3,stu000013@trincoll.edu,STU000013,Quinn,2,2.49,True,English,Williams,2023-10-08
4,stu000021@trincoll.edu,STU000021,Alex,1,2.17,True,Art,Brown,2023-05-07
...,...,...,...,...,...,...,...,...,...
195,stu000978@trincoll.edu,STU000978,Avery,2,2.29,True,History,Brown,2021-08-08
196,stu000983@trincoll.edu,STU000983,Jordan,4,2.02,True,Chemistry,Brown,2022-10-28
197,stu000991@trincoll.edu,STU000991,Taylor,2,2.17,True,English,Garcia,2022-09-06
198,stu000992@trincoll.edu,STU000992,Avery,2,2.04,True,Art,Miller,2020-10-01


In [53]:
from sqlalchemy import text


# Define your SQL query
sql_query = text("SELECT first_name, last_name, major, gpa, is_active FROM students WHERE gpa < 2.5 AND is_active = 1")

# Use pandas to read the SQL query into a DataFrame
df = pd.read_sql_query(sql_query, engine)

# Print the DataFrame as a table
df

,first_name,last_name,major,gpa,is_active
0,Jordan,Smith,English,2.45,1
1,Avery,Smith,Computer Science,2.44,1
2,Avery,Garcia,English,2.43,1
3,Quinn,Williams,English,2.49,1
4,Alex,Brown,Art,2.17,1
...,...,...,...,...,...
195,Avery,Brown,History,2.29,1
196,Jordan,Brown,Chemistry,2.02,1
197,Taylor,Garcia,English,2.17,1
198,Avery,Miller,Art,2.04,1


## Enrollment Counts

In [75]:
pd.merge(
    course_df,
    enrollment_df,
    on='course_id',
    how='inner'
)[['course_name','department','student_id','course_id']].groupby(by='course_id')['student_id'].count().sort_values(ascending=False).head(10)
# .sort_values(by=)

course_id
MATH-3828    51
CS-1477      49
BUS-2431     48
CS-2617      47
BUS-4715     47
ENG-4446     46
HIST-1380    45
CS-2349      45
HIST-1208    45
BIO-1750     44
Name: student_id, dtype: int64

In [76]:
import requests
import json
from pprint import pprint

# Base URL for your API
BASE_URL = "http://localhost:8000"

def test_endpoint(endpoint, params=None):
    """Test an API endpoint and print results"""
    url = f"{BASE_URL}{endpoint}"
    
    try:
        response = requests.get(url, params=params)
        print(f"\n🔍 Testing: {endpoint}")
        print(f"Status Code: {response.status_code}")
        
        if response.status_code == 200:
            data = response.json()
            print("✅ SUCCESS!")
            
            # Pretty print first few results if it's a list
            if isinstance(data, list) and len(data) > 0:
                print(f"Found {len(data)} results. First result:")
                pprint(data[0])
            else:
                pprint(data)
        else:
            print(f"❌ ERROR: {response.status_code}")
            print(response.text)
            
    except requests.exceptions.ConnectionError:
        print("❌ CONNECTION ERROR: Make sure your API is running!")
        print("Run: uvicorn main:app --reload")
    except Exception as e:
        print(f"❌ ERROR: {e}")

def main():
    """Run all API tests"""
    print("🚀 Testing Campus Data API")
    print("=" * 50)
    
    # Test 1: Health check
    test_endpoint("/health")
    
    # Test 2: Root endpoint
    test_endpoint("/")
    
    # Test 3: Get students with filters
    test_endpoint("/students", {"limit": 3, "active_only": True})
    
    # Test 4: Get students by major
    test_endpoint("/students", {
        "major": "Computer Science", 
        "limit": 5,
        "min_gpa": 3.0
    })
    
    # Test 5: At-risk students
    test_endpoint("/students/at-risk", {"gpa_threshold": 2.5})
    
    # Test 6: Student analytics
    test_endpoint("/analytics/students")
    
    # Test 7: Course analytics
    test_endpoint("/analytics/courses", {"department": "CS"})
    
    # Test 8: Get specific student (you might need to change this ID)
    test_endpoint("/students/STU000001")
    
    # Test 9: ML prediction
    test_endpoint("/ml/predictions/student-success/STU000001")
    
    print("\n🎉 All tests completed!")
    print("\n💡 Pro tip: Visit http://localhost:8000/docs for interactive testing!")

# if __name__ == "__main__":
#     main()
main()

🚀 Testing Campus Data API

🔍 Testing: /health
Status Code: 200
✅ SUCCESS!
{'database': 'connected',
 'status': 'healthy',
 'timestamp': '2025-09-08T01:21:55.287159'}

🔍 Testing: /
Status Code: 200
✅ SUCCESS!
{'docs': '/docs',
 'endpoints': {'analytics': '/analytics/students',
               'at-risk': '/students/at-risk',
               'courses': '/analytics/courses',
               'predictions': '/ml/predictions',
               'students': '/students'},
 'message': 'Welcome to Trinity College Campus Data API'}

🔍 Testing: /students
Status Code: 200
✅ SUCCESS!
Found 3 results. First result:
{'email': 'stu000001@trincoll.edu',
 'enrollment_date': '2021-03-12T00:00:00',
 'first_name': 'Jordan',
 'gpa': 2.45,
 'is_active': True,
 'last_name': 'Smith',
 'major': 'English',
 'student_id': 'STU000001',
 'year_level': 2}

🔍 Testing: /students
Status Code: 200
✅ SUCCESS!
Found 5 results. First result:
{'email': 'stu000020@trincoll.edu',
 'enrollment_date': '2022-05-06T00:00:00',
 'first_nam